## Необходимые зависимости

In [13]:
!pip install sentence_transformers
!pip install xlwt
!git clone https://github.com/RusinDmitry/Identifying-implicit-relationships-in-abstract-text

fatal: destination path 'Identifying-implicit-relationships-in-abstract-text' already exists and is not an empty directory.


## Загрузка данных

In [14]:
import pandas as pd
import joblib
from typing import List
from sentence_transformers import SentenceTransformer, util
import torch
import re

#изначально функция для n-gramm, но используется просто для форматирования
def create_ngrams(lst, n):
    return [' '.join(lst[i:i+n]) for i in range(len(lst)-n+1)]

#удалить все кроме даты
def filter_data(text):
    match = re.findall(r'\d{4}-\d{2}-\d{2}', text)
    if len(match) > 0:
        return match[0]
    else:
        return ''

#удалить все кроме чисел и убрать .00
def filter_price(text):
    match = re.findall(r'([0-9\s.,]+)', text)
    if len(match) > 0:
        price = ' '.join(match)
        price = re.sub('\D', '', price)
        if len(price) > 2:
            price = price[:-2]
        return price
    else:
        return ''

#удалить "производитель:, разработчик" перед названием и теги
def filter_maker(text):
    match = re.findall(r':.+', text)
    if len(match) > 0:
        return filter_tags(match[0][1:])
    else:
        return ''

def filter_tags(text):
    text = re.sub('&[A-zА-я ]+;', '', text)
    text = re.sub('[^A-zА-я0-9 ]+', '', text)
    return text

#вариант для csv файла, в котором объединены applic и meta (есть all_test_df.csv и all_train_df.csv)
file_path = '/content/Identifying-implicit-relationships-in-abstract-text/all_train_df.csv'
data = pd.read_csv(file_path)["Data"].iloc[:]
data = data.apply(lambda x: x.split("\n")).to_list()

#вариант для dat файла (в applic файле мало информации о цене, производителяих итд)
#def read_dat_file(file_path):
#    with open(file_path, 'r') as file:
#        lines = file.readlines()
#
#    data = []
#    current_data = []
#    for line in lines:
#        if line.strip() == "*$*":
#            if current_data:
#                data.append(current_data)
#            current_data = []
#        else:
#            current_data.append(line.strip())
#
#    if current_data:
#        data.append(current_data)
#
#    return data
#file_path = '/content/Identifying-implicit-relationships-in-abstract-text/all_applic.dat'
#data = read_dat_file(file_path)

result = pd.DataFrame(columns = ["Контингент (индификатор)",
                                 "Название проекта (Описание)",
                                 "Дата выхода",
                                 "Цена",
                                 "Производитель, разработчик",
                                 "Покупатель, заказчик",
                                 "Участник торгов"])




filters = {"Дата выхода": filter_data,
           "Производитель, разработчик": filter_maker,
           "Цена": filter_price,
           "Покупатель, заказчик": filter_maker,
           "Участник торгов": filter_maker}



## Загрузка модели вложений предложений

In [15]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
pretrained_model = SentenceTransformer('distiluse-base-multilingual-cased-v1', device = DEVICE)

cuda


## Преобразование предложений из тендоров в вложения

In [16]:
import numpy as np

lens = list(map(len, data))
cut_dot = np.add.accumulate(lens)[:-1]

corpus = sum([sum([create_ngrams(data[i], n) for n in range(1, 2)], []) for i in range(len(data))], [])
embeddings = pretrained_model.encode(corpus, convert_to_tensor=False)
embeddings_list = np.split(embeddings, cut_dot)

## Загрузка модели машинного обучения - парсера

In [17]:
model = joblib.load('/content/Identifying-implicit-relationships-in-abstract-text/model.pkl')

In [18]:
def parse(tender: List[str], embedding):
    inform = {}
    for string, sentence in zip(tender, embedding):
        predict = model.predict(sentence.reshape(1, -1))[0]
        if predict in list(result.columns):
            if predict in filters.keys():
                pre_string = filters[predict](string)
            else:
                pre_string = string

            if predict in inform.keys() and predict != "Дата выхода" and predict != "Цена":
                inform[predict] += ', ' + pre_string
            else:
                inform[predict] = pre_string
    return inform


for data_i, embeddings_i in zip(data, embeddings_list):
    new_row = parse(data_i, embeddings_i)
    new_df = pd.DataFrame(new_row, index=[0])
    result = pd.concat([result, new_df], ignore_index=True)

result.to_csv("table.csv")
result.to_excel("table.xls")
result

<ipython-input-18-2908eb1d969a>:24: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  result.to_excel("table.xls")


,Контингент (индификатор),Название проекта (Описание),Дата выхода,Цена,"Производитель, разработчик","Покупатель, заказчик",Участник торгов
0,(Контингент: CGN-20230223005),Результаты строительства гидроэлектроэнергии и...,2023-07-24,36270410,NaN,"Чайна инжиниринг лимитед , Чайна инжинирин...",NaN
1,NaN,Результаты тайфуновой перестройки на заводе MX...,2022-09-16,14010201,NaN,Китайская ядерная промышленность 23 строитель...,NaN
2,NaN,"BOH, BBX/BBC/BBD/BBY, BCM, BBJ, Проект резолюц...",2023-01-31,106022021,NaN,"Китайская ядерная промышленность, Чайна инж...",NaN
3,NaN,"Циркуляр результатов, Наблюдение за осуществле...",2022-12-15,97392204,NaN,"ООО Чжэцзян , Чайна инжиниринг лимитед",NaN
4,(Контингент: CGN-20210615002),В рамках проекта &lt; &lt; Ветроэнергия для бе...,2021-08-03,40818858,Hong Kong,Чайна инжиниринг лимитед,Китайская компания по производству новой энергии
...,...,...,...,...,...,...,...
1543,"(Контингент: CGN-20230628005), Номер: CGN-2023...","Результаты, Надзор за осуществлением данного т...",2023-08-11,,NaN,Чайна инжиниринг лимитед,Чайна Хиросима
1544,(Контингент: CGN-20220820003),150 МВт наземный ветроэнергетический проект по...,2022-10-18,11564367,NaN,", Чайна инжиниринг лимитед",ЧайноЯпонский ветроэнергетик
1545,(Конференция: CGN-20211028003),LF-проект LOT52Ba (переработка вторичной фильт...,2022-07-19,9980000,Shanghai Daily Water Technologies,"Чайна инжиниринг лимитед , Чайна инжинирин...",NaN
1546,(Контингент: CGN-20220729003),Надзор за осуществлением данного тендерного пр...,2022-10-18,2587250,Hyundai Electric Company,Чайна инжиниринг лимитед,Шанхайская компания Синьхуа
